In [ ]:
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text Vectorization
import gensim
from gensim.models import Word2Vec, FastText
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.utils import simple_preprocess

# Modeling
from sklearn.metrics.pairwise import cosine_similarity
from gensim.similarities import WmdSimilarity

# 01 Load Dataset

# 02 Text Vectorization

# 03 Labelling & Similarity Mapping

# 04 Recommendation Modeling

### Comparison

## Saving Vectorization & Similarity Results